# Crimson Folder Watcher - Quick Example

A simple demonstration of the folder watcher functionality.

## Setup

The example files are already prepared in the `./source` directory:
- [my_post.md](./source/my_post.md) - A markdown file
- [any_script.py](./source/any_script.py) - A Python script

In [1]:
from crimson.folder_watcher import create_watcher
import re

# Create watcher
watcher = create_watcher(
    source_dir="./source", 
    output_dir="./output",
    include=["*.py", "*.md"],
    exclude=["*_test.py"],
    process_existing=False,
    auto_start=False
)

In [2]:
# Register Python file editor - add timestamp
def add_timestamp_to_python(src_path, content):
    import datetime
    timestamp = datetime.datetime.now().strftime("# Last modified: %Y-%m-%d %H:%M:%S\n")
    
    if "# Last modified:" in content.split("\n")[0]:
        lines = content.split("\n")
        lines[0] = timestamp.strip()
        return "\n".join(lines)
    else:
        return timestamp + content

watcher.register_editor("*.py", add_timestamp_to_python)

In [3]:
# Register Markdown file editor - convert to stories
def md_to_story_editor(src_path, content):
    titlePrefix = 'AutoPost'
    componentPath = '../components/Page'
    postPath = '../../post' 

    path_parts = src_path.split('/')
    file_name = path_parts[-1].replace('.md', '')
    normalized_file_name = re.sub(r'[^a-zA-Z0-9]', '_', file_name)
    title_path = '/'.join([titlePrefix] + path_parts[:-1])
    md_path = postPath + src_path

    story_content = f"""
// Auto-generated story file for {src_path}
import React from 'react';
import {{ Page }} from '{componentPath}';
import markdownContent from '{md_path}?raw';

export default {{
    title: '{title_path}',
    parameters: {{}},
}};

export const {normalized_file_name} = () => {{
    return Page(markdownContent);
}};
"""

    return story_content, src_path.replace('.md', '.stories.tsx')

watcher.register_editor("*.md", md_to_story_editor)

In [4]:
# Start watching
watcher.start()

Watching './source' for changes...


In [5]:
# Process existing files
watcher.process_existing_files()

Processing existing files...
Edited and saved './source/my_post.md' to './output/my_post.stories.tsx'
Edited and saved './source/any_script.py' to './output/any_script.py'
Processed 2 existing files.


The watcher is now monitoring files. Any changes to `.py` or `.md` files in the source directory will be automatically processed according to our registered editors.

In [6]:
# Stop watching when done
watcher.stop()

Stopped watching for changes.


## Results

Check the output files:
- [any_script.py](./output/any_script.py) - Python file with timestamp added
- [my_post.stories.tsx](./output/my_post.stories.tsx) - Markdown converted to Storybook story